In [1]:
# import pandas as pd
# df = pd.read_csv('/content/drive/MyDrive/Projects/Llama2_Garden/gardening_dataset.csv')
# df.to_parquet('/content/drive/MyDrive/Projects/Llama2_Garden/gardening_dataset.parquet')

In [2]:
cd /content/drive/MyDrive/Projects/Llama2_Garden

/content/drive/MyDrive/Projects/Llama2_Garden


In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7 transformers==4.36.1

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    Conversation
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"
# model_name = "/content/drive/MyDrive/Projects/Llama2_Garden/llama-2-7b-chat.Q8_0.gguf"

# The instruction dataset to use
dataset_name = "/content/drive/MyDrive/Projects/Llama2_Garden/Dataset"

# Fine-tuned model name
new_model = "Yggdrasil-L27BC-V2"

# ################################################################################
# # QLoRA parameters
# ################################################################################

# # LoRA attention dimension
# lora_r = 64

# # Alpha parameter for LoRA scaling
# lora_alpha = 16

# # Dropout probability for LoRA layers
# lora_dropout = 0.1

# ################################################################################
# # bitsandbytes parameters
# ################################################################################

# # Activate 4-bit precision base model loading
# use_4bit = True

# # Compute dtype for 4-bit base models
# bnb_4bit_compute_dtype = "float16"

# # Quantization type (fp4 or nf4)
# bnb_4bit_quant_type = "nf4"

# # Activate nested quantization for 4-bit base models (double quantization)
# use_nested_quant = False

# ################################################################################
# # TrainingArguments parameters
# ################################################################################

# # Output directory where the model predictions and checkpoints will be stored
# output_dir = "./results"

# # Number of training epochs
# num_train_epochs = 1

# # Enable fp16/bf16 training (set bf16 to True with an A100)
# fp16 = False
# bf16 = False

# # Batch size per GPU for training
# per_device_train_batch_size = 4

# # Batch size per GPU for evaluation
# per_device_eval_batch_size = 4

# # Number of update steps to accumulate the gradients for
# gradient_accumulation_steps = 1

# # Enable gradient checkpointing
# gradient_checkpointing = True

# # Maximum gradient normal (gradient clipping)
# max_grad_norm = 0.3

# # Initial learning rate (AdamW optimizer)
# learning_rate = 2e-4

# # Weight decay to apply to all layers except bias/LayerNorm weights
# weight_decay = 0.001

# # Optimizer to use
# optim = "paged_adamw_32bit"

# # Learning rate schedule
# lr_scheduler_type = "cosine"

# # Number of training steps (overrides num_train_epochs)
# max_steps = -1

# # Ratio of steps for a linear warmup (from 0 to learning rate)
# warmup_ratio = 0.03

# # Group sequences into batches with same length
# # Saves memory and speeds up training considerably
# group_by_length = True

# # Save checkpoint every X updates steps
# save_steps = 0

# # Log every X updates steps
# logging_steps = 25

# ################################################################################
# # SFT parameters
# ################################################################################

# # Maximum sequence length to use
# max_seq_length = None

# # Pack multiple short examples in the same input sequence to increase efficiency
# packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [6]:
# # Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")

# # Load tokenizer and model with QLoRA configuration
# compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=use_4bit,
#     bnb_4bit_quant_type=bnb_4bit_quant_type,
#     bnb_4bit_compute_dtype=compute_dtype,
#     bnb_4bit_use_double_quant=use_nested_quant,
# )

# # Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     device_map=device_map
# )
# model.config.use_cache = False
# model.config.pretraining_tp = 1

# # Load LLaMA tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# # Load LoRA configuration
# peft_config = LoraConfig(
#     lora_alpha=lora_alpha,
#     lora_dropout=lora_dropout,
#     r=lora_r,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# # Set training parameters
# training_arguments = TrainingArguments(
#     output_dir=output_dir,
#     num_train_epochs=num_train_epochs,
#     per_device_train_batch_size=per_device_train_batch_size,
#     gradient_accumulation_steps=gradient_accumulation_steps,
#     optim=optim,
#     save_steps=save_steps,
#     logging_steps=logging_steps,
#     learning_rate=learning_rate,
#     weight_decay=weight_decay,
#     fp16=fp16,
#     bf16=bf16,
#     max_grad_norm=max_grad_norm,
#     max_steps=max_steps,
#     warmup_ratio=warmup_ratio,
#     group_by_length=group_by_length,
#     lr_scheduler_type=lr_scheduler_type,
#     report_to="tensorboard"
# )

# # Set supervised fine-tuning parameters
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=packing,
# )

# # Train model
# trainer.train()

# # Save trained model
# trainer.model.save_pretrained(new_model)

In [7]:
# del base_model
# del pipe
torch.cuda.empty_cache()
# del trainer
# import gc
# gc.collect()
# gc.collect()

In [8]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [9]:
pipe = pipeline(task="conversational", model=model, tokenizer=tokenizer, max_new_tokens=750)

In [10]:
def get_reply(prompt, conversation):
  conversation.add_message(message = {
      "role": "user",
      "content": prompt
  })
  conversation = pipe(conversation)
  result = conversation.messages[-1]["content"]
  return result

In [11]:
conversation = Conversation([{
    "role": "system",
    "content": """You are a gardening assistant bot. Your primary task is to answer questions based STRICTLY on gardening.

      RULES:
      - ONLY answer if the question relates directly to the gardening.
      - Ensure the answers are relevant and helpful in the context of gardening.
      - If the question is a greeting (like "Hello", "Hi", etc.), respond with a corresponding greeting."""}])
while True:
  prompt = input("User: ")
  if prompt == "exit":
    print("Thank you for gardening with us")
    break
  res = get_reply(prompt, conversation)
  print(f"Assistant: ",res,"\n\n")

User: how to climb a tree



No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Assistant:   Hello! Climbing a tree can be a fun and rewarding experience, but it's important to do it safely. Here are some steps you can follow to climb a tree:

1. Choose a tree that is sturdy and has branches that are easy to grab onto. Look for trees with a wide trunk and branches that are spaced out evenly.
2. Make sure you have the right equipment. You'll need a sturdy ladder or a rope and harness. If you're using a ladder, make sure it's sturdy and securely positioned.
3. Start by climbing up the ladder or rope. Make sure you're securely attached to the ladder or rope and that you're not going to slip.
4. Once you're at the top of the ladder or rope, take a moment to look around and make sure you're safe. Check for any branches that are loose or weak, and make sure you're not going to fall.
5. Now you can start climbing the tree. Make sure you're holding onto the ladder or rope at all times, and that you're not going to slip.
6. As you climb, look for branches that are sturdy a

In [12]:
# questions =[
#     "how to grow potatoes",
#     "how to grow basil",
#     "How to play cricket",
#     "How to climb a tree"
# ]
# for q in questions:
#   print("User: ", q)
#   res = get_reply(q, conversation)
#   print("Assistant: ", res, "\n\n")

User:  how to grow potatoes


KeyboardInterrupt: 

In [13]:
conversation

Conversation id: db8518ed-6840-43ae-a061-7ff348a05b46
system: You are a gardening assistant bot. Your primary task is to answer questions based STRICTLY on gardening.

      RULES:
      - ONLY answer if the question relates directly to the gardening.
      - Ensure the answers are relevant and helpful in the context of gardening.
      - If the question is a greeting (like "Hello", "Hi", etc.), respond with a corresponding greeting.
user: how to climb a tree
assistant:  Hello! Climbing a tree can be a fun and rewarding experience, but it's important to do it safely. Here are some steps you can follow to climb a tree:

1. Choose a tree that is sturdy and has branches that are easy to grab onto. Look for trees with a wide trunk and branches that are spaced out evenly.
2. Make sure you have the right equipment. You'll need a sturdy ladder or a rope and harness. If you're using a ladder, make sure it's sturdy and securely positioned.
3. Start by climbing up the ladder or rope. Make sure y

In [ ]:
!nvidia-smi